Since dirctly download csv file does not work well (the Chinese characters can not be recognized). I tried download xlsx file, and load it as df, however, the df does not work either. At last, I download xlsx, save it as csv (choose UTF-8), it works.

Remember to download the data to desktop first and change the name (too long).

1C is the raw data. 2C is saved as csv. 3C is mannualy changed the question 3 4 5.

In [2]:
import pandas as pd
import numpy as np
import re

The raw data is downloaded at June 6th, 2023. I made it to cleaned data based on the method I used in 5255 final project (the same to the US data). I used some specific method because of the Chinese characters

In [41]:
# Load raw data
raw = pd.read_csv('China/3C_raw_345.csv')

In [42]:
# Delete the first row (remain the 0 row) so that the data has only one header row
cleaned = raw.iloc[1:].copy()

# Convert the 'StartDate' column to a datetime format
# cleaned['StartDate'] = pd.to_datetime(cleaned['StartDate'])

# Filter the DataFrame to keep only the rows with dates on or after July 1, 2019. I did not use this.
# cleaned = cleaned[cleaned['StartDate'] >= '2019-06-30']

# Convert the 'Q51' column to a numeric data type
cleaned['Q51'] = pd.to_numeric(cleaned['Q51'], errors='coerce')

# Filter the DataFrame to remove rows where 'Q51' is greater than 30 or less than 17
cleaned = cleaned[(cleaned['Q51'] <= 30) & (cleaned['Q51'] >= 17)].dropna(subset=['Q51'])

# First, ensure that the missing values are represented as NaN
cleaned[['3', 'Q4', 'Q5']] = cleaned[['3', 'Q4', 'Q5']].apply(pd.to_numeric, errors='coerce')

# Then drop the rows where all three columns are NaN
cleaned = cleaned.dropna(subset=['3', 'Q4', 'Q5'], how='all')

# Then remove the rows where all three columns are 0
cleaned = cleaned[~((cleaned['3'] == 0) & (cleaned['Q4'] == 0) & (cleaned['Q5'] == 0))]

# Save the cleaned DataFrame to the 'data' folder
cleaned.to_csv('China/C_cleaned_data.csv', index=False, encoding='utf-8-sig')


In [43]:
# Load the cleaned data 
cleaned = pd.read_csv('China/C_cleaned_data.csv')

Add a colunm called covid, the cases after March 2020 are marked 1, the cases before it are marked 0

In [44]:
# Convert 'StartDate' to datetime if not already
cleaned['StartDate'] = pd.to_datetime(cleaned['StartDate'])

# Create 'covid' column
cleaned['covid'] = cleaned['StartDate'].apply(lambda x: 1 if x >= pd.to_datetime('2020-03-01') else 0)

# save it to .csv file (for checking the data)
cleaned.to_csv('China/C_cleaned2.csv', index=False, encoding='utf-8-sig')

X: PCS,PSF,PSM
indicators for them:
S1-4 (Q77_1,Q77_2,Q77_3,Q77_4),
F1-10 (Q50_1_1,Q50_1_3,Q50_1_4,Q50_1_6,Q50_1_7,Q50_1_9,Q50_1_10,Q50_1_11,Q50_1_13,Q50_1_14), 
M1-10 (Q50_2_1,Q50_2_3,Q50_2_4,Q50_2_6,Q50_2_7,Q50_2_9,Q50_2_10,Q50_2_11,Q50_2_13,Q50_2_14)

Y: RRS
indicators for it:R1-5 (Q76_1,Q76_2,RRS3,Q76_4,Q76_5)

M: AA,including secure(Q20_1), fearful(Q20_2), preoccupied(Q20_3), dimissing(Q20_4) 

covariates: rela_sta(Q6),age(Q51),gender(1),SES(Q69),duration(Q9),covid(?)

In [45]:
# Load the cleaned data 
cleaned2 = pd.read_csv('China/C_cleaned2.csv')

# Create a new DataFrame called 'ssea' with some columns from the cleaned data
ssea = cleaned2[['Q77_1', 'Q77_2', 'Q77_3', 'Q77_4', 'Q50#1_1', 'Q50#1_3', 'Q50#1_4', 'Q50#1_6', 'Q50#1_7', 
                'Q50#1_9', 'Q50#1_10', 'Q50#1_11', 'Q50#1_13', 'Q50#1_14', 'Q50#2_1', 'Q50#2_3', 'Q50#2_4', 
                'Q50#2_6', 'Q50#2_7', 'Q50#2_9', 'Q50#2_10', 'Q50#2_11', 'Q50#2_13', 'Q50#2_14', 'Q76_1', 'Q76_2', 
                'Q76_3', 'Q76_4', 'Q76_5', 'Q20_1', 'Q20_2', 'Q20_3', 'Q20_4', 'Q6', 'Q51', '1', 'Q69', 'Q9', 'covid']]

# Rename the columns to the variable names
ssea.columns = ['S1', 'S2', 'S3', 'S4', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'M1', 
                'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'R1', 'R2', 'R3', 'R4', 'R5', 
                'sec', 'fea', 'pre', 'dis', 'rela_sta', 'age', 'gender', 'SES', 'duration', 'covid']

# Output the first few rows of 'ssea' to make sure it looks correct
print(ssea.head())


    S1   S2   S3   S4   F1   F2   F3   F4   F5   F6  ...  sec  fea  pre  dis  \
0  6.0  6.0  6.0  6.0  4.0  2.0  3.0  3.0  3.0  4.0  ...  7.0  5.0  4.0  3.0   
1  3.0  4.0  2.0  3.0  2.0  2.0  2.0  3.0  4.0  3.0  ...  7.0  1.0  1.0  2.0   
2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  4.0  4.0  4.0  4.0   
3  2.0  2.0  5.0  4.0  2.0  1.0  2.0  3.0  2.0  1.0  ...  6.0  2.0  2.0  2.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  5.0  5.0  5.0  5.0   

   rela_sta   age  gender  SES  duration  covid  
0         2  21.0       6    3       一个月      0  
1         2  21.0       2    3         4      0  
2         2  21.0       6    3        28      0  
3         2  22.0       2    3        2年      0  
4         1  20.0       6    2         1      0  

[5 rows x 39 columns]


In [46]:


# save it to .csv file (for checking the data)
ssea.to_csv('China/C_ssea_without_deleting.csv', index=False, encoding='utf-8-sig')

delete rows that have missing value in certain columns

In [47]:
# Load the data 
ssea2 = pd.read_csv('China/C_ssea_without_deleting.csv')



# Drop the rows where all elements are missing.
ssea2 = ssea2.dropna(how='all')

# Select the first 24 columns and the last 4 columns
first_24_columns = ssea2.iloc[:, :24]
selected_columns = ssea2.iloc[:, 29:33]

# Drop the rows where all elements are missing in the first 24 columns and the last 4 columns.
ssea2 = ssea2.loc[~first_24_columns.isnull().all(axis=1) & ~selected_columns.isnull().all(axis=1)]


# Save the cleaned data to a new .csv file
ssea2.to_csv('China/C_ssea_dropna.csv', index=False, encoding='utf-8-sig')


manage rela_sta

In [48]:
# Load the data 
ssea3 = pd.read_csv('China/C_ssea_dropna.csv')

# change 'rela_sta' to make it only have values 0 and 1

def relationship_transform(value):
    if pd.isna(value):
        return np.nan
    else:
        value_set = set(map(int, value.split(',')))
        if value_set.intersection({2, 4, 5, 7}):
            return 1
        else:
            return 0

# Create 'rela_sta' based on the 'Q6' column in cleaned_data
ssea3['rela_sta'] = ssea3['rela_sta'].apply(relationship_transform)


# Save the cleaned data to a new .csv file
ssea3.to_csv('China/C_ssea_rela.csv', index=False, encoding='utf-8-sig')

manage gender. Chinese survey has some coding issue. Need to recode first.

In [49]:
# Load the data 
ssea4 = pd.read_csv('China/C_ssea_rela.csv')

# Define a dictionary for the recode
recode_dict = {2:2, 3:3, 4:4, 5:5, 7:6, 6:1}

# Apply the recode to the 'Gender' column
ssea4['gender'] = ssea4['gender'].replace(recode_dict)

# Create a new column 'gender_transformed' by combining the certain categories into 1
ssea4['gender_trans'] = ssea4['gender'].apply(lambda x: 3 if x in [3, 4, 5, 6] else x if x in [1, 2] else np.nan)

# Create dummy variables for 'gender'
dummies = pd.get_dummies(ssea4['gender_trans'], prefix='gender',drop_first=True)

# Concatenate the original DataFrame and the dummy DataFrame along columns
ssea4 = pd.concat([ssea4, dummies], axis=1)

# Save the cleaned data to a new .csv file
ssea4.to_csv('China/C_ssea_gender.csv', index=False, encoding='utf-8-sig')

In [50]:
# after checking gender and it's dummy coding varibales, delating them
ssea4 = ssea4.drop(['gender', 'gender_trans'], axis=1)

# Save the cleaned data to a new .csv file
ssea4.to_csv('China/C_ssea_gender2.csv', index=False, encoding='utf-8-sig')

copy 'China/C_ssea_gender2.csv' and manage duration manualy. 

In [61]:
# load the data
ssea5 = pd.read_csv('China/C_ssea_duration.csv')

scaleing

In [62]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Create a list of columns to scale
columns_to_scale = ['S1', 'S2', 'S3', 'S4', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10','sec','fea','pre','dis'
,'age','SES','duration']
# Copy your dataframe
ssea_scaled = ssea5.copy()

# Standardization
scaler = StandardScaler()
ssea_scaled[columns_to_scale] = scaler.fit_transform(ssea5[columns_to_scale])


In [63]:
ssea_scaled.to_csv('China/C_ssea_scaled.csv', index=False)


In [64]:
# load the data
ssea6 = pd.read_csv('China/C_ssea_scaled.csv')

In [65]:
# Now, replace NaN back to -999, as it's required for the Mplus model
ssea6 = ssea6.fillna(-999)

In [66]:
# Save the cleaned data to a new .dat file, remove the first row (header)
ssea6.to_csv('China/C_ssea.dat', sep=' ', index=False, header=False)

create mean

In [2]:
import pandas as pd

# Load your data
ssea7 = pd.read_csv('China/C_ssea_scaled.csv')

# Define the columns that belong to each scale
pcs_cols = ['S4', 'S1', 'S2', 'S3']
rrs_cols = ['R5', 'R1', 'R2', 'R3', 'R4']
psf_cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10']
psm_cols = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10']

# Compute the mean for each scale
ssea7['PCS'] = ssea7[pcs_cols].mean(axis=1)
ssea7['RRS'] = ssea7[rrs_cols].mean(axis=1)
ssea7['PSF'] = ssea7[psf_cols].mean(axis=1)
ssea7['PSM'] = ssea7[psm_cols].mean(axis=1)

# Save the new DataFrame to a new .csv file
ssea7.to_csv('China/C_ssea_means.csv', index=False)


In [3]:
# load the data
ssea8 = pd.read_csv('China/C_ssea_means.csv')

# Now, replace NaN back to -999, as it's required for the Mplus model
ssea8 = ssea8.fillna(-999)

# Save the cleaned data to a new .dat file, remove the first row (header)
ssea8.to_csv('China/C_ssea_mean.dat', sep=' ', index=False, header=False)

In [5]:
# load the data
ssea9 = pd.read_csv('China/C_ssea_scaled.csv')

# Create a list of columns to scale
columns_to_scale = ['R1', 'R2', 'R3', 'R4', 'R5']

# Copy your dataframe
ssea_scaled2 = ssea9.copy()

# Standardization
scaler = StandardScaler()
ssea_scaled2[columns_to_scale] = scaler.fit_transform(ssea9[columns_to_scale])


In [6]:
ssea_scaled2.to_csv('China/C_ssea_RRSscaled.csv', index=False)

In [9]:
# load the data
ssea10 = pd.read_csv('China/C_ssea_RRSscaled.csv')

# Now, replace NaN back to -999, as it's required for the Mplus model
ssea10 = ssea10.fillna(-999)

# Save the cleaned data to a new .dat file, remove the first row (header)
ssea10.to_csv('China/C_ssea_RRSscaled.dat', sep=' ', index=False, header=False)

Do not scale, create the unscaled data

In [10]:
# load the data
ssea11 = pd.read_csv('China/C_ssea_duration.csv')
# Now, replace NaN back to -999, as it's required for the Mplus model
ssea11 = ssea11.fillna(-999)

# Save the cleaned data to a new .dat file, remove the first row (header)
ssea11.to_csv('China/C_ssea_unscaled.dat', sep=' ', index=False, header=False)

MIN MAX scale

In [11]:
# load the data
ssea12 = pd.read_csv('China/C_ssea_duration.csv')

In [12]:
from sklearn.preprocessing import MinMaxScaler

# Create a list of columns to scale
columns_to_scale = ['S1', 'S2', 'S3', 'S4', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'M1', 'M2', 'M3', 
                    'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10','R1', 'R2', 'R3', 'R4','R5','sec','fea','pre','dis'
,'age','SES','duration']

# Copy your dataframe
ssea_scaled = ssea12.copy()

# Min-Max Scaling
scaler = MinMaxScaler()
ssea_scaled[columns_to_scale] = scaler.fit_transform(ssea12[columns_to_scale])

# save
ssea_scaled.to_csv('China/C_ssea_minmax.csv', index=False)

In [13]:
# load the data
ssea13 = pd.read_csv('China/C_ssea_minmax.csv')
# Now, replace NaN back to -999, as it's required for the Mplus model
ssea13 = ssea13.fillna(-999)

# Save the cleaned data to a new .dat file, remove the first row (header)
ssea13.to_csv('China/C_ssea_minmax.dat', sep=' ', index=False, header=False)